# Convert EDF to expipe files
The visual EEG inspection table 'eeg-sichtung.xlsx' has been manually translated into a form which is more easy to read automatically.
The table is located in the new file 'data_annotation.ods'.
It contains one row per day an animal is recorded.
Relevant metadata such as animal id, recording channel, artifacts or missing data are stored in separate columns.

This notebooks loads the original edf files sequentially and converts the recordings to the expipe format [1].
Here, each day an animal is recorded is its own action.

[1] https://github.com/CINPLA/expipe

In [1]:
import numpy as np
import pandas as pd
from tempfile import mkdtemp
import zipfile
import shutil
from tqdm import tqdm
import expipe
import pyedflib
import datetime
import os
import tools
from format_data_annotation import dict_format
import pdb

In [2]:
path = ''
fname = 'data_annotation.ods'

In [3]:
# Info about the EEG files
path_edf = "/home/jovyan/work/raw_files/"
path_expipe = "/home/jovyan/work/data_expipe/epimirna"

# decide whether existing data should be rebuild or not
bool_keepexisting = False

### Read data annotation

In [4]:
t = pd.read_excel(path+fname, engine="odf", dtype=dict_format)

### Create Expipe project

In [5]:
animals=t['ID'].unique()
print(animals)

[ 1218  1227  1237  1243  1251  1254  1257  1260  1261  1262  1263  1264
  1265  1269  1270  1271  1274  1275  1276   322   323   324   325   326
   328   329  3210  3211  3212  3215  3216  3218 32220  3222 32223  3224
  3226  3227  3228  3231  3232  3233  3235  3237  3238  3239  3240  3241
  3242  3243  3244  3245  3246  3247  3248  3251  3253  3254  3256  3258
  3259  3260  3261  3262  3263  3266  3267  3268  3269  3270  3271  3272
  3273  3274  3275  3276  3277  3278  3279  3280  3281  3282  3283  3284
  3285]


In [6]:
# Open project
project = expipe.require_project(path_expipe)

for animal_i in animals:
    
    # Register each animal
    entity_i = project.require_entity(str(animal_i))
    t_i = t[t['ID']==animal_i]
    entity_i.modules['info'] = {
        'channel': t_i['Channel'].unique()[0],
        'machine': t_i['Machine'].unique()[0]
    }
    # Create action for each animal
    name_action_i = 'record_'+str(animal_i)
    action_i = project.require_action(name_action_i)
    
    action_i.type = "eeg recording"
    action_i.entities = [str(animal_i)]
    
    # store sub-table for each animal
    fname_annot = 'df_data_annotation.pckl'
    t_i.to_pickle(action_i.data_path().joinpath(fname_annot))
    action_i.data['df_data_annotation'] = fname_annot

### Load EEG files and store them as actions

#### Exclude entries with overlapping recordings


In [7]:
t = t[t['Conflict overlap recording']!='x']

#### Test consistency of leftover table

In [8]:
t = t[t['Device off'].isnull() & t['Missing data'].isnull()]
# Test consistency of the table for data extraction
tools.test_consistency_of_annotation_table(t)

In [9]:
t = t[t['Missing data']!='x']

In [10]:
# Open EDF files by date and machine
grp = t.groupby(['Date', 'Machine'])
for (date_i, machine_i), grp_i in tqdm(grp):
    
    # unzip file to temporary folder
    date_str = date_i.date().strftime('%Y-%m-%d')
    fname = "EDF_" + machine_i + '_' + date_str
    fname_zip = fname + '.zip'
    tmpdir = mkdtemp()
    try:
        with zipfile.ZipFile(path_edf + fname_zip, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)
    except FileNotFoundError as err:
        print(err)
        shutil.rmtree(tmpdir)
        continue

    # load recordings
    fname_edf = fname + '.edf'
    try:
        f = pyedflib.EdfReader(tmpdir + '/' + fname_edf)
    except OSError as err:
        print("OS error: {0}".format(err))
        print(fname_edf + ' not found')
        f.close()
        shutil.rmtree(tmpdir)
        continue
    
    # extract information from recording
    n = f.signals_in_file
    start_time = f.getStartdatetime()
    signal_labels = f.getSignalLabels()
    nsamples = f.getNSamples()
    assert len(np.unique(nsamples)) == 1
    nsamples = np.unique(nsamples)[0]
    
    # associate recording to animal entity
    for label_i in signal_labels:
        pos_i = signal_labels.index(label_i)
        t_ii = grp_i[grp_i['Channel'] == int(label_i)]
        if np.any(t_ii):
            animal_id = t_ii['ID'].unique()
            assert(len(animal_id) == 1)
            animal_id = str(animal_id[0])
            
            # get corresponding action
            action_i = project.require_action('record_'+str(animal_id))
            
            name_rec = 'eeg_recording_' + date_str
            # check if recording already exists
            if (name_rec in list(action_i.data) and
                bool_keepexisting):
                print('Skipped file. Already present: ' + name_rec)
                continue
            else:
                # Get all information from table except date
                info = t_ii.to_dict('records')[0]
                
                # date is removed here, it is stored in a different place
                del info['Date']
                
                # if any entry in info is datetime.time, convert to hours, minutes, seconds
                for k, v in info.items():
                    if isinstance(v, datetime.time):
                        info[k] = v.strftime("%d/%m/%Y, %H:%M:%S")
                
                sampling_freq = f.getSampleFrequency(pos_i)
                info['sampling_freq'] = sampling_freq
                info['nsamples'] = nsamples
                info['duration'] = nsamples*1./sampling_freq
                info['dimension'] = f.getPhysicalDimension(pos_i)
                #info['starttime'] = start_time.strftime("%d/%m/%Y, %H:%M:%S")
                #info['annotations'] = list(t_ii.to_dict('index').values())

                mod_info = action_i.require_module(name=name_rec+'_info', contents=info)

                 # link file in expipe
                action_i.data[name_rec] = name_rec + '.npz'

                # store recordings in separate file
                sig_ii = f.readSignal(pos_i)
                action_data_path = str(action_i.data_path(name_rec))
                np.savez_compressed(action_data_path, eeg=sig_ii)
    # close EDF file
    f.close()
    # delete temporary folder
    shutil.rmtree(tmpdir)


  3% 26/1023 [04:59<2:34:52,  9.32s/it]

[Errno 2] No such file or directory: '/home/jovyan/work/raw_files/EDF_yolo_2014-08-16.zip'
[Errno 2] No such file or directory: '/home/jovyan/work/raw_files/EDF_yolo_2014-08-17.zip'


  4% 46/1023 [08:34<3:13:17, 11.87s/it]

[Errno 2] No such file or directory: '/home/jovyan/work/raw_files/EDF_yolo_2014-09-05.zip'
[Errno 2] No such file or directory: '/home/jovyan/work/raw_files/EDF_yolo_2014-09-06.zip'
[Errno 2] No such file or directory: '/home/jovyan/work/raw_files/EDF_yolo_2014-09-07.zip'


  7% 73/1023 [18:54<6:46:44, 25.69s/it] 

...

 47% 485/1023 [6:24:59<8:15:45, 55.29s/it] 

OS error: /tmp/tmpy63f84et/EDF_yolo_2015-12-02.edf: can not open file, no such file or directory
EDF_yolo_2015-12-02.edf not found


 48% 486/1023 [6:25:05<6:01:51, 40.43s/it]

OS error: /tmp/tmpjvg2z6vk/EDF_yolo_2015-12-03.edf: can not open file, no such file or directory
EDF_yolo_2015-12-03.edf not found


100% 1023/1023 [14:49:24<00:00, 52.17s/it] 


In [44]:
a = t.groupby(['ID']).first()
g, n = np.unique(a['group'], return_counts=True)

In [ ]:
annotation = {
    '1': 'FF_EPG_5Hz',
    '2': 'FF_EPG_130Hz',
    '3': 'PP_EPG_5Hz',
    '4': 'PP_EPG_130Hz',
    '5': 'DG_EPG_5Hz',
    '6': 'DG_EPG_130Hz',
    '8': 'FF_ME_5Hz',
    '9': 'FF_ME_130Hz',
    '10': 'PP_EPG_5Hz',
    '11': 'PP_EPG_130Hz',
    '12': 'DG_EPG_5Hz',
    '13': 'DG_EPG_130Hz',
    '14': 'FF_noDBS',
    '15': 'VHC_noDBS',
    '16': ,
    'pps_only': 'PP_noDBS'
    'vhc_epilep_1 Hz': ,
    'vhc_epilep_HF': ,
    'vhc_epilep_LF (5 Hz)': ,
    'vhc_epileptog_LF': ,
    'vhc_mE_Kontrolle': ,
    'vhc_mE_LF (1 Hz)': ,
    'vhc_mE_LF (1Hz)': ,
    'vhc_mE_LF (5 Hz)': ,
    'vhc_mE_LF (5Hz)'
    



In [46]:
list(zip(g,n))

[('1', 6),
 ('10', 3),
 ('11', 4),
 ('12', 2),
 ('13', 3),
 ('14', 3),
 ('2', 2),
 ('3', 8),
 ('4', 7),
 ('5', 4),
 ('6', 7),
 ('8', 2),
 ('9', 1),
 ('?', 1),
 ('pps_only', 19),
 ('vhc_epilep_1 Hz', 2),
 ('vhc_epilep_HF', 2),
 ('vhc_epilep_LF (5 Hz)', 1),
 ('vhc_epileptog_LF', 1),
 ('vhc_mE_Kontrolle', 3),
 ('vhc_mE_LF (1 Hz)', 1),
 ('vhc_mE_LF (1Hz)', 1),
 ('vhc_mE_LF (5 Hz)', 1),
 ('vhc_mE_LF (5Hz)', 1)]

In [47]:
g

array(['1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '8',
       '9', '?', 'pps_only', 'vhc_epilep_1 Hz', 'vhc_epilep_HF',
       'vhc_epilep_LF (5 Hz)', 'vhc_epileptog_LF', 'vhc_mE_Kontrolle',
       'vhc_mE_LF (1 Hz)', 'vhc_mE_LF (1Hz)', 'vhc_mE_LF (5 Hz)',
       'vhc_mE_LF (5Hz)'], dtype=object)

85